# 4.8 Grouping Data and Aggregating Variables

### This script incluse the following points:

#### 01. Importing Libraries and Loading Flies
#### 02. Aggregating Mean of order_number
#### 03. Creating a Loyalty Flag with transform() and loc()
#### 04. Analyzing Spending Habits of Loyalty Flag
#### 05. Targeting Different Types of Spenders
#### 06. Measuring Customer Frequency
#### 07. Exporting Data

### 01. Importing Libraries and Loading Files

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

# Creating path variable for loading data
path = r'C:\Users\widne\Documents\CareerFoundry Exercises\Data_Immersion\Achievement 4\03-2024 Instacart Basket Analysis'

# Loading orders_products_merged_derived pickle
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

### 02. Aggregating Mean of order_number

In [2]:
# Checking dataframe
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders


In [3]:
# Splitting the data into groups based on department_id
# Applying agg() function to each group to get the mean values of order_number
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


When aggregating the mean over the entire dataframe we get more departments grouped than the subset. Also, the overall means of each department are higher in the whole dataframe when compared to the subset.

### 03. Creating a Loyalty Flag with transform() and loc()

In [4]:
# Splitting data into groups by user_id
# Applying transform() function on order_number to generate max orders for each user
# Creating new column max_order which contains the result of the aggregation
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\widne\AppData\Local\Temp\ipykernel_13520\1553656957.py:4: FutureWarning: The provided callable <function max at 0x0000015BD5BC5440> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [5]:
# Checking dataframe for new column max_order
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders,10
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders,10
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders,10
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders,10


In [6]:
# Creating loyalty_flag column based on customers max_order
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
# Checking freqency of loyalty_flag values
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

### 04. Analyzing Spending Habits of Loyalty Flag

In [10]:
# Grouping data by loyalty_flag
# Aggregating the mean, min, and max values of each loyalty_flag
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### 05. Targeting Different Types of Spenders

In [11]:
# Splitting data into groups by user_id
# Applying transform() function on prices to genearate the average price for each user
# Creating new column average_spend which contains the result of the appregation
ords_prods_merge['average_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\widne\AppData\Local\Temp\ipykernel_13520\4130575393.py:4: FutureWarning: The provided callable <function mean at 0x0000015BD5BC5E40> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [12]:
# Checking dataframe for new column
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Least busy,Most orders,10,New customer,6.367797


In [13]:
# Creating spender_flag column based on customers average spend
ords_prods_merge.loc[ords_prods_merge['average_spend'] < 10, 'spender_flag'] = 'Low spender'

In [14]:
ords_prods_merge.loc[ords_prods_merge['average_spend'] >= 10, 'spender_flag'] = 'High spender'

In [15]:
# Checking frequency of spender_flag column
ords_prods_merge['spender_flag'].value_counts(dropna = False)

spender_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

### 06. Measuring Customer Frequency

In [16]:
# Splitting data into groups by user_id
# Applying transform() function on days_since_prior_order to generate the median time between orders for each user
# Creating new column median_frequency which contains the result of the aggregation
ords_prods_merge['median_frequency'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

C:\Users\widne\AppData\Local\Temp\ipykernel_13520\1766514479.py:4: FutureWarning: The provided callable <function median at 0x0000015BD5D05D00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_frequency'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [17]:
# Checking dataframe for new column
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend,spender_flag,median_frequency
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,both,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,False,196,1,1,...,7,9.0,both,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,False,196,1,1,...,7,9.0,both,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,False,196,1,1,...,7,9.0,both,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,False,196,1,1,...,7,9.0,both,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5


In [18]:
# Creating frequency_flag column based on customers shopping frequency
ords_prods_merge.loc[ords_prods_merge['median_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['median_frequency'] > 10) & (ords_prods_merge['median_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['median_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [21]:
# Checking value counts of frequency_flag column
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

### 07. Exporting Data

In [22]:
# Exporting dataframe as pickle
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))